In [87]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 30)

In [88]:
# Not real training set, was playing around in csv file and added "on_point" column
training_data = pd.read_csv(r"C:\Users\Brandon\Desktop\Home_Depot_Data\train_01.csv", encoding="ISO-8859-1")

In [89]:
training_data.dtypes

id                 int64
product_uid        int64
product_title     object
search_term       object
relevance        float64
on_point          object
dtype: object

In [90]:
g = training_data.columns.to_series().groupby(training_data.dtypes).groups
g

{dtype('int64'): [u'id', u'product_uid'],
 dtype('float64'): [u'relevance'],
 dtype('O'): [u'product_title', u'search_term', u'on_point']}

In [91]:
training_data.head(10)

,id,product_uid,product_title,search_term,relevance,on_point
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,yay
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,yay
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00,yay
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33,nah
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67,yay
5,18,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,convection otr,3.00,yay
6,20,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,microwave over stove,2.67,yay
7,21,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,microwaves,3.00,yay
8,23,100007,Lithonia Lighting Quantum 2-Light Black LED Em...,emergency light,2.67,yay
9,27,100009,House of Fara 3/4 in. x 3 in. x 8 ft. MDF Flut...,mdf 3/4,3.00,yay


In [92]:
training_data.search_term[2]

u'deck over'

In [93]:
from sklearn.feature_extraction.text import CountVectorizer

In [94]:
# Instantiate the vectorizer with n-grams of length one or two
CountV = CountVectorizer(ngram_range=(1,2))

# Vector where each row is bag-of-words for a single quote
X = CountV.fit_transform(training_data.search_term)
print X

  (0, 2897)	1
  (0, 4566)	1
  (0, 2901)	1
  (1, 4566)	1
  (2, 7514)	1
  (2, 17665)	1
  (2, 7533)	1
  (3, 19798)	1
  (3, 21965)	1
  (3, 12535)	1
  (3, 19811)	1
  (3, 21999)	1
  (4, 21965)	1
  (4, 17312)	1
  (4, 9640)	1
  (4, 22008)	1
  (4, 17314)	1
  (5, 6789)	1
  (5, 17438)	1
  (5, 6791)	1
  (6, 17665)	1
  (6, 16174)	1
  (6, 23600)	1
  (6, 16186)	1
  (6, 17678)	1
  :	:
  (74064, 12201)	1
  (74064, 15363)	1
  (74064, 21321)	1
  (74064, 8716)	1
  (74064, 14348)	1
  (74064, 21330)	1
  (74064, 22132)	1
  (74064, 15374)	1
  (74064, 22133)	1
  (74064, 12202)	1
  (74064, 8720)	1
  (74065, 11294)	1
  (74065, 7587)	1
  (74065, 27741)	1
  (74065, 27742)	1
  (74065, 11301)	1
  (74066, 7297)	1
  (74066, 13640)	1
  (74066, 21803)	1
  (74066, 9954)	1
  (74066, 1879)	1
  (74066, 9957)	1
  (74066, 21805)	1
  (74066, 7298)	1
  (74066, 1880)	1


In [95]:
# Create an array where each element encodes whether the array is Relevant or Not
Y = (training_data.on_point == 'yay').values.astype(np.int)
print Y

[1 1 1 ..., 0 1 0]


In [96]:
from sklearn.cross_validation import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X, Y)

In [97]:
# vector of all quotes
yay_CV = CountV.fit(training_data.search_term)

# a few helper functions
def accuracy_report(_clf):
    print "Accuracy: %0.2f%%" % (100 * _clf.score(xtest, ytest))

    #Print the accuracy on the test and training dataset
    training_accuracy = _clf.score(xtrain, ytrain)
    test_accuracy = _clf.score(xtest, ytest)

    print "Accuracy on training data: %0.2f" % (training_accuracy)
    
# a function to run some tests
def AnalyzeReview(testquote, _clf):
    print "\""  + testquote + "\" is judged by clasifier to be..."
    testquote = yay_CV.transform([testquote])

    if (_clf.predict(testquote)[0] == 1):
        print "... on point."
    else:
        print "... not on point."
    return(_clf.predict(testquote)[0])

In [98]:
# Multinomial NB and accuracy report

from sklearn.naive_bayes import MultinomialNB

print "MultinomialNB"
clf_mn = MultinomialNB().fit(xtrain, ytrain)
accuracy_report(clf_mn)

MultinomialNB
Accuracy: 66.54%
Accuracy on training data: 0.75


In [99]:
# Bernoulli NB and accuracy report

from sklearn.naive_bayes import BernoulliNB
print "BernoulliNB"
clf_bern = BernoulliNB().fit(xtrain, ytrain)
accuracy_report(clf_bern)

BernoulliNB
Accuracy: 66.50%
Accuracy on training data: 0.75


In [101]:
# Logistic Regression for comparison

from sklearn.linear_model import LogisticRegression
print "Logistic Regression"
clf_lr = LogisticRegression().fit(xtrain, ytrain)
accuracy_report(clf_lr)

Logistic Regression
Accuracy: 68.09%
Accuracy on training data: 0.77


In [103]:
AnalyzeReview("microwaves", clf_mn)
AnalyzeReview("microwaves", clf_bern)
AnalyzeReview("rain shower head", clf_lr)

"microwaves" is judged by clasifier to be...
... on point.
"microwaves" is judged by clasifier to be...
... on point.
"rain shower head" is judged by clasifier to be...
... on point.


1